In [65]:


import pandas as pd
import numpy as np
from model_team14 import select_features

## select imputed & transformed data
X=pd.read_csv('../data/X_data_tr.csv', index_col='date', parse_dates=True)
y=pd.read_csv('../data/y_data_tr.csv', index_col='date', parse_dates=True)
threshold=0.3
criteria=None

df_feature=pd.read_csv('../data/df_feature.csv')

selected_features=list(df_feature[df_feature.select==1]['variable'])

y_type = 'y_agg'
test_year = 8

X_train=X[selected_features][:-(test_year*12)]
y_train=y[y_type][:-(test_year*12)]           
X_test=X[selected_features][-(test_year*12):]   
y_test=y[y_type][-(test_year*12):]

In [66]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)
# Timesteps will define how many Elements we have
TIME_STEPS = 5

# X_train, y_train = create_dataset(X_train, y_train, TIME_STEPS)
# X_test, y_test = create_dataset(X_test, y_test, TIME_STEPS)

# print(X_train.shape)

In [68]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

def grid_search(X_train, y_train, X_test, y_test, epochs=[15,20],batch_sizes=[12,64],time_steps=[1,5]):
    results = []

    for time_step in time_steps:
        X_train, y_train = create_dataset(X_train, y_train, time_step)
        X_test, y_test = create_dataset(X_test, y_test, time_step)
        for num_epochs in epochs:
            for batch_size in batch_sizes:
                # create the model
                model = Sequential()
                model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
                model.add(Dense(1, activation='sigmoid'))
                model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])
                model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)
                print(model.summary())
                # Final evaluation of the model
                scores = model.evaluate(X_test, y_test, verbose=0)
                auc = scores[1]*100
                print("AUC: %.2f%%" % (auc))
                results.append({
                    'time_steps': time_step,
                    'num_epochs': num_epochs,
                    'batch_size': batch_size,
                    'AUC': auc
                })

    return results

grid_search(X_train, y_train, X_test, y_test)

TypeError: unsupported operand type(s) for -: 'int' and 'list'